# Sparse Autoencoder Training Demo

## Setup

In [1]:
# Autoreload
%load_ext autoreload
%autoreload 2

In [13]:
from sparse_autoencoder import TensorActivationStore, SparseAutoencoder
from sparse_autoencoder.train.pipeline import pipeline
from sparse_autoencoder.source_data.pile_uncopyrighted import PileUncopyrightedDataset
from transformer_lens import HookedTransformer
from transformer_lens.utils import get_device
from transformers import GPT2TokenizerFast
import torch
import wandb

In [3]:
print(torch.__version__)
device = get_device()

1.13.1+cu117


### Source Dataset

In [4]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
source_data = PileUncopyrightedDataset(tokenizer=tokenizer)

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

### Source Model

In [5]:
src_model = HookedTransformer.from_pretrained("solu-1l", dtype="float32")
src_d_mlp = src_model.cfg.d_mlp
src_d_mlp

(…)_1L512W_C4_Code/resolve/main/config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model_final.pth:   0%|          | 0.00/213M [00:00<?, ?B/s]

(…)igits/resolve/main/tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

(…)nizer-digits/resolve/main/tokenizer.json:   0%|          | 0.00/2.04M [00:00<?, ?B/s]

(…)its/resolve/main/special_tokens_map.json:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

Loaded pretrained model solu-1l into HookedTransformer


2048

### Activation Store

In [6]:
max_items = 2_000_000
store = TensorActivationStore(max_items, src_d_mlp, device)

In [20]:
src_dataloader = create_src_dataloader(
    "NeelNanda/c4-code-tokenized-2b",
    collate_fn=collate_neel_c4_tokenized,
    shuffle_buffer_size=10_000,
    random_seed=0,
    batch_size=64,
)

NameError: name 'create_src_dataloader' is not defined

### Autoencoder

In [7]:
autoencoder = SparseAutoencoder(src_d_mlp, src_d_mlp * 8, torch.zeros(src_d_mlp))
autoencoder

SparseAutoencoder(
  (encoder): Sequential(
    (0): TiedBias(position=pre_encoder)
    (1): ConstrainedUnitNormLinear(in_features=2048, out_features=16384, bias=True)
    (2): ReLU()
  )
  (decoder): Sequential(
    (0): ConstrainedUnitNormLinear(in_features=16384, out_features=2048, bias=False)
    (1): TiedBias(position=post_decoder)
  )
)

## Training

If you initialise [wandb](https://wandb.ai/site), the pipeline will automatically log all metrics to wandb.

In [12]:
wandb.init(project="sae", name="Alan_Run_2", dir=".cache/wandb")

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7efdcc84f010>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7efd505df9a0, raw_cell="wandb.init(project="sae", name="Alan_Run_2", dir=".." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bvast_cold/root/sparse_autoencoder/demo.ipynb#X21sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

wandb: WARNING Path .cache/wandb/wandb/ wasn't writable, using system temp directory.


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


wandb: WARNING Path .cache/wandb/wandb/ wasn't writable, using system temp directory


In [14]:
pipeline(
    src_model=src_model,
    src_model_activation_hook_point="blocks.0.mlp.hook_post",
    src_model_activation_layer=0,
    src_dataloader=source_data,
    activation_store=store,
    num_activations_before_training=max_items,
    autoencoder=autoencoder,
    device=device,
)

Generate/Train Cycles: 0it [00:00, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (3180 > 1024). Running this sequence through the model will result in indexing errors


AttributeError: 'dict' object has no attribute 'shape'

In [19]:
a = next(iter(source_data))
torch.LongTensor(a["input_ids"]).shape

torch.Size([250])